#LangChain Exercise:

In this exercise, you’ll take your first steps with LangChain.

You’ll learn how to:
1. Build a simple chain that connects a prompt and a model.
2. Load documents from a text file.
3. Transform documents into smaller chunks so they can be processed by an LLM.
4. By the end, you’ll combine everything into a workflow that can summarize documents step by step.


In [ ]:
#@title Environment Variables
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Install packages

In [ ]:
%pip install "langchain==0.3.27" -qqq
%pip install "langchain-openai==0.3.35" -qqq
%pip install "langchain-community==0.3.31" -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 41.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have request

# Model (OpenAI)

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

# Part 1: Create a Custom Chain

1. Import:
```
from langchain_core.prompts import PromptTemplate
```
2. Create a PromptTemplate that asks the model:
    - “Summarize this text in one sentence: {input_text}”
3. Combine the template and the model (ChatOpenAI) into a chain.
4. Test your chain with a short text.

In [ ]:
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize this text in one sentence:\n\n{input_text}"
)

# Chain
chain = prompt | model

# Test
test_result = chain.invoke({"input_text": "LangChain helps developers build applications powered by LLMs."})
print("Chain test output:", test_result.content)

Chain test output: LangChain enables developers to create applications that utilize large language models (LLMs).


# Part 2: Document Loading

1. Create a text file in Colab (sample.txt) with at least 3 paragraphs.
    - Example: ask chatGPT for 3 paragraphs about the Portuguese history.
2. Load the file using TextLoader.

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/content/Transactions.txt")

docs = loader.load()

print(docs)

[Document(metadata={'source': '/content/Transactions.txt'}, page_content='Transactions\nTransactions let users spend satoshis. Each transaction is constructed out of several parts which enable both simple direct payments and complex transactions.\n\nIntroduction\nThis section will describe each part and demonstrate how to use them together to build complete transactions.\n\nTo keep things simple, this section pretends coinbase transactions do not exist. Coinbase transactions can only be created by Bitcoin miners and they’re an exception to many of the rules listed below. Instead of pointing out the coinbase exception to each rule, we invite you to read about coinbase transactions in the block chain section of this guide.\n\nThe Parts Of A Transaction\nThe Parts Of A Transaction\n\nThe figure above shows the main parts of a Bitcoin transaction. Each transaction has at least one input and one output. Each input spends the satoshis paid to a previous output. Each output then waits as an U

# Part 3: Document Loading

1. Use RecursiveCharacterTextSplitter to split the document into chunks of ~150 characters with 30 overlap.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=30)
splits = splitter.split_documents(docs)

print("Number of chunks:", len(splits))
print("\nChunks:\n", splits)

Number of chunks: 114

Chunks:
 [Document(metadata={'source': '/content/Transactions.txt'}, page_content='Transactions'), Document(metadata={'source': '/content/Transactions.txt'}, page_content='Transactions let users spend satoshis. Each transaction is constructed out of several parts which enable both simple direct payments and complex'), Document(metadata={'source': '/content/Transactions.txt'}, page_content='direct payments and complex transactions.'), Document(metadata={'source': '/content/Transactions.txt'}, page_content='Introduction\nThis section will describe each part and demonstrate how to use them together to build complete transactions.'), Document(metadata={'source': '/content/Transactions.txt'}, page_content='To keep things simple, this section pretends coinbase transactions do not exist. Coinbase transactions can only be created by Bitcoin miners and'), Document(metadata={'source': '/content/Transactions.txt'}, page_content='created by Bitcoin miners and they’re an exce

# Part 4: Chain + Transformation

- Run your chain from Part 1 on each chunk.
- Collect the summaries in a list.
- Print the list of summaries.

In [ ]:
summaries = [] # initialize the summaries list
for chunk in splits:
    result = chain.invoke({"input_text": chunk.page_content})
    summaries.append(result.content)

print("Chunk Summaries:")
for i, summary in enumerate(summaries, 1):
    print(f"{i}. {summary}")

Chunk Summaries:
1. The text discusses transactions, likely covering their definition, types, and significance in various contexts.
2. Transactions allow users to spend satoshis through various components that facilitate both simple payments and more complex financial interactions.
3. The text refers to financial operations involving immediate payments and intricate financial exchanges.
4. The introduction outlines the components involved and instructions on how to combine them for building complete transactions.
5. This section simplifies the discussion by excluding coinbase transactions, which are exclusively generated by Bitcoin miners.
6. Bitcoin miners create exceptions to many established rules, particularly concerning the coinbase transaction.
7. For detailed information on Coinbase transactions, please refer to the blockchain section of this guide.
8. The text outlines the various components that make up a transaction.
9. A Bitcoin transaction consists of at least one input and

# 💡 Extra Challenge (Optional)
After summarizing each chunk, use a chain that takes all the chunk summaries and produces a final overall summary of the document.

In [ ]:
combined_summaries = "".join(summaries) # join the summaries in one single string the LLM can use

final_summary = chain.invoke({"input_text": combined_summaries})

print("\nFinal Overall Summary:\n", final_summary.content)